# 15-12-30

David Rowe gave the following explanation for his centroid finding in PS3: (**My comments in bold**)

"
I have a routine SimplePhotometry(X, Y, ApRadius, Image, Stats) that calculates the centroid, flux, background, and a number of other statistics from Image, given a location on the image (X,Y) and the radius of the aperture (ApRadius). The calculated statistics are contained in the structure called Stats. For peak tracking I call this routine 4 times using the previously calculated values of the centroid as the new location. In other words, the routine is as follows:

Given Xm, Ym as the mouse location

SimplePhotometry(Xm,Ym,ApRadius,Image,Stats)
for i = 0 to 3
	SimplePhotometry(Stats.Xcen,Stats.Ycen,ApRadius,Image,Stats)
next 

(Xcen,Ycen) is the calculated centroid.

** Centroid calculated for mouse location, then centroid is calculated several more times with centroid as center of calculation, within circular aperture**

This causes the circular aperture to be better and better centered on the peak in the image. Why so many iterations? For empirical reasons. I found that five total calls was needed in marginal cases.

I've included the VB.Net code for SimplePhotometry() if you're interested.

I hope this answers your question. If not, let me know.

My best,
Dave


    Public Shared Sub SimplePhotometry(ByVal X As Double, ByVal Y As Double, ByVal ApRad As Double, ByVal ImData As ImageLib.ImageType, ByRef Stats As StarStatsType)

        ' calculate the statistics for star located at pixel location X,Y in image ImData
        ' results are returned in Stats
        ' ApRad is the radius of the object aperture, in pixels
        ' the background is not considered

        Dim Ip0, Ip1, Jp0, Jp1 As Integer
        Dim Val As Double
        Dim S, Sx, Sy, Sx2, Sy2, Sxy, Arg, MaxVal As Double
        Dim Xcen, Ycen, R, R2 As Double
        Dim X2, Y2, Xy, R2x, R2y, Sr2X, Sr2Y As Double
        Dim Theta, u2, V2, AR, Sigma As Double
        Dim Px, Py As Integer
        Dim N As Integer
        Dim B, Den As Double

** Center of centroid region defined by mouse location ** 

        Xcen = X
        Ycen = Y

** Size of input image saved **

        Px = ImData.N1
        Py = ImData.N2

        'aperture stats

** Calculating pixel indices surrounding the aperture to be looped through **

        Ip0 = Math.Max(0, Math.Round(Xcen - (ApRad + 0.5)))
        Ip1 = Math.Min(Px - 1, Math.Round(Xcen + (ApRad + 0.5)))
        Jp0 = Math.Max(0, Math.Round(Ycen - (ApRad + 0.5)))
        Jp1 = Math.Min(Py - 1, Math.Round(Ycen + (ApRad + 0.5)))
        
        ' first find centroid
        S = 0 : Sx = 0 : Sy = 0
        
** Loop through square of incices containing the aperture **
        
        For i = Ip0 To Ip1
            For j = Jp0 To Jp1
            
** Check that present pixel is within radius of aperture **            
         
                R = Math.Sqrt((Xcen - i) ^ 2 + (Ycen - j) ^ 2)
                If R <= ApRad Then
                    Val = ImData.Data(i, j)
                    
** Accumulating values for centroid calculation **
                    
                    S = S + Val
                    Sx = Sx + Val * i
                    Sy = Sy + Val * j
                End If
            Next j
        Next i

** Calculating centroid values **

        If S > 0 Then
            Xcen = Sx / S
            Ycen = Sy / S
        Else
            Xcen = 0
            Ycen = 0
        End If

** Assign calculated centroid **

        Stats.Xcen = Xcen
        Stats.Ycen = Ycen
        Stats.Total = S
        ' find moments about this centroid
        N = 0 : S = 0 : Sx2 = 0 : Sy2 = 0 : Sxy = 0 : X2 = 0 : Y2 = 0 : Xy = 0 : Sr2X = 0 : Sr2Y = 0 : MaxVal = 0
        For i = Ip0 To Ip1
            For j = Jp0 To Jp1
                R = Math.Sqrt((Xcen - i) ^ 2 + (Ycen - j) ^ 2)
                If R <= ApRad Then
                    Val = ImData.Data(i, j)
                    MaxVal = Math.Max(MaxVal, Val)
                    S = S + Val
                    
** Calculating Central Moments **

*mu_10*

                    Sx2 = Sx2 + Val * (i - Xcen) ^ 2
*mu_01*
                    
                    Sy2 = Sy2 + Val * (j - Ycen) ^ 2
*Mu_11*                    
                    
                    Sxy = Sxy + Val * (i - Xcen) * (j - Ycen)
** Not sure what the rest of this is for **                    
                    
                    Sr2X = Sr2X + Val * (i - Xcen) * ((i - Xcen) ^ 2 + (j - Ycen) ^ 2)
                    Sr2Y = Sr2Y + Val * (j - Ycen) * ((i - Xcen) ^ 2 + (j - Ycen) ^ 2)
                    N = N + 1
                End If
            Next j
        Next i

        If S > 0 Then
            X2 = Sx2 / S
            Y2 = Sy2 / S
            Xy = Sxy / S
            R2x = Sr2X / S
            R2y = Sr2Y / S
            If (X2 + Y2) > 0 Then
                R2x = R2x / (X2 + Y2) ^ 1.5
                R2y = R2y / (X2 + Y2) ^ 1.5
            Else
                R2x = 0
                R2y = 0
            End If
        End If

        Arg = X2 + Y2
        If Arg > 0 Then Stats.RMS = Math.Sqrt((X2 + Y2) / 2.0) Else Stats.RMS = 0

        Theta = Math.Atan2(2 * Xy, X2 - Y2) / 2.0
        u2 = X2 * Math.Cos(Theta) ^ 2 + Y2 * Math.Sin(Theta) ^ 2 + 2 * Xy * Math.Sin(Theta) * Math.Cos(Theta)
        V2 = Y2 * Math.Cos(Theta) ^ 2 + X2 * Math.Sin(Theta) ^ 2 - 2 * Xy * Math.Sin(Theta) * Math.Cos(Theta)
        If u2 > 0 And V2 > 0 Then AR = Math.Sqrt(u2 / V2) Else AR = 0

        Stats.AR = AR
        Stats.Theta = Theta
        Stats.U2 = u2
        Stats.V2 = V2
        Stats.X2 = X2
        Stats.Y2 = Y2
        Stats.Xy = Xy
        Stats.R2x = R2x
        Stats.R2y = R2y
        Stats.Max = MaxVal

        ' -------------------------------------------------------------------------
        ' fit radial profile to a Gaussian

        Ip0 = Math.Max(0, Math.Round(Xcen - (ApRad + 0.5)))
        Ip1 = Math.Min(Px - 1, Math.Round(Xcen + (ApRad + 0.5)))
        Jp0 = Math.Max(0, Math.Round(Ycen - (ApRad + 0.5)))
        Jp1 = Math.Min(Py - 1, Math.Round(Ycen + (ApRad + 0.5)))

        S = 0 : Sx = 0 : Sy = 0 : N = 0 : Sxy = 0 : Sx2 = 0
        For i = Ip0 To Ip1
            For j = Jp0 To Jp1
                R2 = (Xcen - i) ^ 2 + (Ycen - j) ^ 2
                R = Math.Sqrt(R2)
                If R <= ApRad Then
                    Val = ImData.Data(i, j)
                    If Val > 0.1 * MaxVal Then
                        Val = Math.Log(Val)
                        N += 1
                        Sy = Sy + Val
                        Sx = Sx + R2
                        Sx2 = Sx2 + R2 * R2
                        Sxy = Sxy + Val * R2
                    End If
                End If
            Next j
        Next i
        B = 0
        Den = Sx * Sx - N * Sx2
        If Den <> 0 Then B = (Sx * Sy - N * Sxy) / Den
        Sigma = 0
        If B < 0 Then Sigma = Math.Sqrt(-1.0 / (2.0 * B))
        Stats.FWHMGauss = 2.35 * Sigma

        '----------------------------------------------------------------------------

        '----------------------------------------------------------------------------
        ' fit to a Moffat Distribution, f(r) = (alfa + beta*r^2)^-MoffExp

        Dim alfa, beta, Rmoff As Double
        Dim MoffExp, MoffExpInv As Double

        MoffExp = 2.5
        MoffExpInv = 1.0 / MoffExp

        S = 0 : Sx = 0 : Sy = 0 : N = 0 : Sxy = 0 : Sx2 = 0
        For i = Ip0 To Ip1
            For j = Jp0 To Jp1
                R2 = (Xcen - i) ^ 2 + (Ycen - j) ^ 2
                R = Math.Sqrt(R2)
                If R <= ApRad Then
                    Val = ImData.Data(i, j)
                    If Val > 0.1 * MaxVal Then
                        Val = (MaxVal / Val) ^ MoffExpInv
                        N += 1
                        Sy = Sy + Val
                        Sx = Sx + R2
                        Sx2 = Sx2 + R2 * R2
                        Sxy = Sxy + Val * R2
                    End If
                End If
            Next j
        Next i
        beta = 0
        Den = Sx * Sx - N * Sx2
        If Den <> 0 Then beta = (Sx * Sy - N * Sxy) / Den
        If N > 0 Then alfa = (Sy - beta * Sx) / N
        If alfa <> 0 Then
            beta = beta / alfa
            alfa = 1.0
        End If
        If beta > 0 Then Rmoff = (1.0 / beta) ^ 0.5
        Stats.FWHMmoff = Rmoff * 2.0 * (2.0 ^ MoffExpInv - 1.0) ^ 0.5
        '---------------------------------------------------------------------------

    End Sub
    
"

I see that Dave's procedure recursively calculates the centroid of the aperture, using the last centroid location as the center of the next search's mask. 

I can imitate this behavior with a combination of my work so far. This would be an automatic solution
1. Find contours of side lobes
2. Calculate centroid of side lobe (masked by contour)
3. Calculate circle mask centered on centroid that fits insie contour
4. Calculate centroid with circle mask
5. Use new centroid location as center to calculate circular mask again
6. Repeat steps 4-5 3x

For a manual solution, I can imitate the PS3 behavior
1. User selects the aperture diameter and location (with mouse or arrow keys)
2. Calculate centroid with circle mask
3. Use new centroid location as center of new circular mask (same diameter as before)
4. Repeat steps 2-3 3x

In [ ]:
# Re-doing my auto centroid finding using the automatic solution explained above

In [ ]:
# Import Modules
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.fftpack import fft2,ifft2,fftshift
from speckle_fns import fits_import, circ_filter1
from speckle_fns import deconv0,deconv1, postprocess
from speckle_fns import filter_lpf, filter_hpf, filter_interference
from speckle_fns import fits_view
import sys
import cv2
import math

%matplotlib inline

In [ ]:
# Folder containing FITS files
fileDirectory = '/home/niels/Documents/FITS/'

# Filenames (without extensions) of the Double/Reference pairs
# Preprocessed files must have _PSD terminations on them in the same folder
filenames=[ ['KP330','KP331'], 
            ['KP332','KP333'], 
            ['KP334','KP335'], 
            ['KP336','KP338'], 
            ['KP339','KP340'],
            ['KP341','KP342'] ]

In [ ]:
# Select which set of data to use:
k = 0

# View sample fits images:
imgDoubleStar = fits_view(fileDirectory+filenames[k][0]+'.fits',0)
imgSingleStar = fits_view(fileDirectory+filenames[k][1]+'.fits',0)

plt.figure(figsize=(15,7))
plt.subplot(1,2,1)
plt.imshow(imgDoubleStar)
plt.title('Double Star Image')
plt.subplot(1,2,2)
plt.imshow(imgSingleStar)
plt.title('Reference Star Image')

# Generating Autocorrelogram
# Import FITS file data
psdDoubleStar = fits_import(fileDirectory+filenames[k][0]+'_PSD.fits').astype(float)
psdSingleStar = fits_import(fileDirectory+filenames[k][1]+'_PSD.fits').astype(float)

plt.figure(figsize=(15,15))
plt.imshow(fftshift(np.log10(psdDoubleStar)))
plt.title("Double Star PSD")

## Perform Deconvolution
constant = 1E-15
imgF = fftshift(deconv1(psdDoubleStar, psdSingleStar, constant))

# Filter PSD:
imgF_filtered = filter_interference(imgF)
imgF_filtered = filter_lpf(imgF_filtered, 10)

# Postprocess
img_autocorr = postprocess(imgF_filtered)

# Plot Image
plt.figure(figsize = (8,8))
plt.imshow(img_autocorr)
plt.title("Autocorr of Filtered Image")
plt.colorbar()


In [ ]:
# Want to find the lowest threshold that we only see the 3 main lobes

# New variable for operating on
autocorr = img_autocorr

# Start thresh at max value of autocorr (assuming that is from central peak)
threshold = np.floor(autocorr.max())

# Calculate values to increment/decrement threshold by depending on height of autocorrelogram
increment_coarse = autocorr.max()/10
increment_fine   = autocorr.max()/100

# Assume we start with 1 contour visible 
num_contours = 1

# Step down threshold until we see the 3 contours we expect
while (num_contours != 3):
    # Decrement the threshold
    threshold = threshold - increment_fine
    
    # Calculate indices in image above threshold
    above_thresh_i = autocorr > threshold
    
    # Create thresholded image
    autocorr_thresh = np.zeros(np.shape(autocorr))
    autocorr_thresh[above_thresh_i] = 255
    autocorr_thresh = autocorr_thresh.astype(np.uint8)
    
    # Find contours of threshold image
    im,contours,hierarchy = cv2.findContours(autocorr_thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    num_contours = len(contours)
    
    if threshold < 0:
        print("Error in finding 3 contours")
        sys.exit()
        
# Calculate indices in image above threshold
above_thresh_i = autocorr > threshold
    
# Create thresholded image
autocorr_thresh = np.zeros(np.shape(autocorr))
autocorr_thresh[above_thresh_i] = 255
autocorr_thresh = autocorr_thresh.astype(np.uint8)
        
#print(threshold)
#plt.imshow(autocorr_thresh)

# Now we've found our 3 main contours. Keep stepping down until we don't see 3 anymore
# Can be less than 3 if they start to blend together, or more than 3 if other noise in
#  image appears
while (num_contours == 3):
    # Decrement the threshold
    threshold = threshold - increment_fine
    
    # Calculate indices in image above threshold
    above_thresh_i = autocorr > threshold
    
    # Create thresholded image
    autocorr_thresh = np.zeros(np.shape(autocorr))
    autocorr_thresh[above_thresh_i] = 255
    autocorr_thresh = autocorr_thresh.astype(np.uint8)
    
    # Find contours of threshold image
    im,contours,hierarchy = cv2.findContours(autocorr_thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    num_contours = len(contours)
    
    if threshold < 0:
        print("Error in moving threshold below 3 contours")
        sys.exit()
        
# Calculate indices in image above threshold
above_thresh_i = autocorr > threshold
    
# Create thresholded image
autocorr_thresh = np.zeros(np.shape(autocorr))
autocorr_thresh[above_thresh_i] = 255
autocorr_thresh = autocorr_thresh.astype(np.uint8)
        
#print(threshold)
#plt.imshow(autocorr_thresh)

# Step back up in threshold until we are back to 3 contours
while (num_contours != 3):
    # Decrement the threshold
    threshold = threshold + increment_fine
    
    # Calculate indices in image above threshold
    above_thresh_i = autocorr > threshold
    
    # Create thresholded image
    autocorr_thresh = np.zeros(np.shape(autocorr))
    autocorr_thresh[above_thresh_i] = 255
    autocorr_thresh = autocorr_thresh.astype(np.uint8)
    
    # Find contours of threshold image
    im,contours,hierarchy = cv2.findContours(autocorr_thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    num_contours = len(contours)    
    
    if threshold > autocorr.max():
        print("Error in moving threshold back up to 3 contours")
        sys.exit()

# Calculate indices in image above threshold
above_thresh_i = autocorr > threshold
    
# Create thresholded image
autocorr_thresh = np.zeros(np.shape(autocorr))
autocorr_thresh[above_thresh_i] = 1
autocorr_thresh = autocorr_thresh.astype(np.uint8)

# Display results
print("Final Threshold = ",threshold)
plt.figure(figsize=(8,8))
plt.imshow(np.multiply(autocorr_thresh,autocorr))

In [ ]:
## Now that we have the contours, we want to find their centroids for sorting purposes
centroid = np.zeros((3,2))

for i in np.arange(3):
    # Calculating Image Moment/Raw Moment of contour
    M = cv2.moments(contours[i])
    
    # Calculate centroid X and Y components 
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    
    centroid[i] = (cy, cx)
    
## Find the central contour
# Center pixel indices
center = np.divide(np.shape(autocorr),2)
center_distance = np.zeros(3)

# Calculate distance from each centroid to center of image
for i in np.arange(3):
    center_distance[i] = np.linalg.norm(centroid[i] - center)

# Get index of central contour as contour with min distance to center
i = np.where(center_distance == center_distance.min())
i_center = i[0]

# Calculate central contour mask
mask_center = np.zeros(np.shape(autocorr))
cv2.drawContours(mask_center,contours,i_center,(1,1,1),-1)

## Finding indices of side lobes
# Possible indices of side lobes
i_side_lobes = [0,1,2]
# Remove the center lobe index
i_side_lobes.remove(i_center)

# Make empty mask images
mask_side = np.zeros((2,np.shape(autocorr)[0],np.shape(autocorr)[1]))
lobe_side = np.zeros((2,np.shape(autocorr)[0],np.shape(autocorr)[1]))

# Create masks and calculate side lobes
for i in np.arange(2):
    cv2.drawContours(mask_side[i],contours,i_side_lobes[i],(1,1,1),-1)
    lobe_side[i] = (np.multiply(autocorr,mask_side[i]))#.astype(np.uint8)

# Display masked side lobes
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.imshow(lobe_side[0])
plt.title("Side Lobe 1")
plt.subplot(1,2,2)
plt.imshow(lobe_side[1])
plt.title("Side Lobe 2")

In [ ]:
## Calculating intensity weighted centroid of side lobes, using entire contour

M00 = np.zeros((2))
M10 = np.zeros((2))
M01 = np.zeros((2))
centroid = np.zeros((2,2))

# Calculate for each side lobe
for lobe in np.arange(2):
    
    # Calculate average
    M00[lobe] = np.sum(lobe_side[lobe])
    
    # Calculate X component
    # Loop through each column
    for column in np.arange(np.shape(lobe_side[lobe])[1]):
        
        M10[lobe] += np.multiply(column,np.sum(lobe_side[lobe,:,column]))
        
        #print('X = ', column, np.multiply(column,np.sum(lobe_side[lobe,:,column])))
    
    # Calculate Y component
    # Loop through each column
    for row in np.arange(np.shape(lobe_side[lobe])[0]):
        
        M01[lobe] += np.multiply(row,np.sum(lobe_side[lobe,row,:]))
    
    # Calculate X and Y Centroid Components of each lobe
    centroid[lobe,0] = M10[lobe]/M00[lobe] # X Component
    centroid[lobe,1] = M01[lobe]/M00[lobe] # Y Component

# Print results of centroid finding
print(M00)
print(M10)
print(M01)
print(centroid[0,0], centroid[0,1])
print(centroid[1,0], centroid[1,1])

# Print original image with centroid locations overlayed
autocorr_cen = np.array(autocorr)
lobe_cen = np.array(lobe_side)

# Put lines on X coordinates of centroids
autocorr_cen[:,np.round(centroid[0,0]).astype(int)] = 0
autocorr_cen[:,np.round(centroid[1,0]).astype(int)] = 0
# Put lines on Y coordinates of centroids
autocorr_cen[np.round(centroid[0,1]).astype(int),:] = 0
autocorr_cen[np.round(centroid[1,1]).astype(int),:] = 0

# Put lines on sidelobe 0
lobe_cen[0,:,np.round(centroid[0,0]).astype(int)] = 0
lobe_cen[0,np.round(centroid[0,1]).astype(int),:] = 0

# Put lines on sidelobe 1
lobe_cen[1,:,np.round(centroid[1,0]).astype(int)] = 0
lobe_cen[1,np.round(centroid[1,1]).astype(int),:] = 0

# Graph all centroid marked plots
plt.figure(figsize=(20,8))

plt.subplot(1,3,1)
plt.imshow(lobe_cen[0])
plt.title("Sidelobe 0 Centroid Location")

plt.subplot(1,3,2)
plt.imshow(lobe_cen[1])
plt.title("Sidelobe 1 Centroid Location")

plt.subplot(1,3,3)
plt.imshow(autocorr_cen)
plt.title("Both Sidelobe Centroid Locations")

In [ ]:
## Finding radius of circle centered on centroid that fits inside contour

lobe_mask_radius = np.zeros(2) # Radius of circles to be calculated 
lobe_mask_radius.fill(math.inf) # Initial value = Infinite

# Calculate for each side lobe
for i in np.arange(2):
    # For each point on contour
    for perimeter_point in contours[i_side_lobes[i]]: 
        # Calculating distance between calculated centroid and present contour point
        distance = np.linalg.norm(perimeter_point - centroid[i])
        # Save the minimum value of previously calculated min radius and present calc
        lobe_mask_radius[i] = min([lobe_mask_radius[i],distance])
     
print(lobe_mask_radius)

In [ ]:
# Recursively find the centroid 3 more times. 
# Use the previously calculated centroid as the center of the circular aperture, 
#  whose diameter was calculated above

# Loop 3 times
for j in np.arange(3):

    # Creating masked images of side lobes
    # Start with blank images
    lobe_side_masked = np.zeros((2,512,512))

    # Creating fully filled in mask
    peak_mask = np.zeros((2,512,512,3))

    # Marking the mask with circles centered on peaks
    cv2.circle(peak_mask[0], (centroid[0,0].astype(np.uint16),centroid[0,1].astype(np.uint16)), lobe_mask_radius[0].astype(np.uint8), (1,1,1), -1)
    cv2.circle(peak_mask[1], (centroid[1,0].astype(np.uint16),centroid[1,1].astype(np.uint16)), lobe_mask_radius[1].astype(np.uint8), (1,1,1), -1)

    # Apply the mask to the autocorrelogram
    lobe_side_masked[0] = peak_mask[0,:,:,0]*lobe_side[0]
    lobe_side_masked[1] = peak_mask[1,:,:,0]*lobe_side[1]
    
    ## Calculating intensity weighted centroid of side lobes, with circular masking

    M00 = np.zeros((2))
    M10 = np.zeros((2))
    M01 = np.zeros((2))
    centroid = np.zeros((2,2))

    # Calculate for each side lobe
    for lobe in np.arange(2):

        # Calculate average
        M00[lobe] = np.sum(lobe_side_masked[lobe])

        # Calculate X component
        # Loop through each column
        for column in np.arange(np.shape(lobe_side_masked[lobe])[1]):

            M10[lobe] += np.multiply(column,np.sum(lobe_side_masked[lobe,:,column]))

        # Calculate Y component
        # Loop through each column
        for row in np.arange(np.shape(lobe_side[lobe])[0]):

            M01[lobe] += np.multiply(row,np.sum(lobe_side_masked[lobe,row,:]))

        # Calculate X and Y Centroid Components of each lobe
        centroid[lobe,0] = M10[lobe]/M00[lobe] # X Component
        centroid[lobe,1] = M01[lobe]/M00[lobe] # Y Component
        
    print("Iteration: ", j)
    print(centroid)


For k = 0: 


Iteration:  0 <br/>
[[ 403.22224796  405.48294385]<br/>
 [ 108.31065958  106.05485897]]<br/>
Iteration:  1<br/>
[[ 403.22224796  405.48294385]<br/>
 [ 108.31065958  106.05485897]]<br/>
Iteration:  2<br/>
[[ 403.22224796  405.48294385]<br/>
 [ 108.31065958  106.05485897]]<br/>
 
These results are fairly similar to the data from 15-12-28_centroids.ipynb: 

k = 0: <br />
Centroids (Sphere centered on peak) <br />
 [[ 403.24772626  405.91946966]<br />
 [ 108.75227374  106.08053034]]<br />
Centroids (Contours) <br />
 [[ 403.37878518  405.93153327]<br />
 [ 108.62121482  106.06846673]]<br />
 
I will say this centroid finding algorithm is complete. Just need to turn this into functions now